In [1]:
import numpy as np
import scipy
from scipy.linalg import expm
import random
import math
from scipy.stats import unitary_group
import matplotlib.pyplot as plt
from scipy.optimize import minimize, curve_fit, basinhopping
import matplotlib
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
import lightning as L
from lightning.pytorch.loggers import TensorBoardLogger
import torch_optimizer

import os
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter

import torch

In [2]:
def set_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    


set_random_seed(42)
L.seed_everything(42)

Seed set to 42


42

In [3]:
torch.cuda.is_available()

True

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [5]:
data = []
for ch in range(4):
    for H in range(3):
        with open(f"Calibration Example_915\ch{ch + 1}_H{H + 1}.txt") as file:
            for line in file:
                try:
                    row = [float(num) for num in line.split()]
                    data.append([H, ch] + row)
                except:
                    pass

def create_un_mat(v, device):

    Mc = torch.zeros((4, 4), dtype=torch.complex128, device=device)

    Mc[0, 1] = v[0] + 1j * v[1]
    Mc[1, 0] = v[0] - 1j * v[1]

    Mc[0, 2] = v[2] + 1j * v[3]
    Mc[2, 0] = v[2] - 1j * v[3]

    Mc[0, 3] = v[4] + 1j * v[5]
    Mc[3, 0] = v[4] - 1j * v[5]

    Mc[1, 2] = v[6] + 1j * v[7]
    Mc[2, 1] = v[6] - 1j * v[7]

    Mc[1, 3] = v[8] + 1j * v[9]
    Mc[3, 1] = v[8] - 1j * v[9]

    Mc[2, 3] = v[10] + 1j * v[11]
    Mc[3, 2] = v[10] - 1j * v[11]

    U_torch = torch.matrix_exp(1j * Mc)
    return U_torch

def f(v, data):

    M1 = create_un_mat(v[:12], device=device)
    M2 = create_un_mat(v[12:24], device=device)

    alpha = v[24:33].reshape(3, 3)
    h_0 = v[33:36]
    cost = torch.tensor(0.0, dtype=torch.float32, device=device)

    for i in data:
        x = torch.zeros(3, dtype=torch.float32, device=device)
        x[i[0]] = 10**(-5) * i[2] ** 2
        y = torch.tensor(i[3:], dtype=torch.complex128, device=device)
        y /= torch.sum(y)

        h_list = h_0 + alpha @ x
        H_diag = torch.cat([
            torch.exp(1j * h_list),
            torch.tensor([1.0], dtype=torch.complex128, device=device)
        ])
        H = torch.diag(H_diag)
        predict = torch.abs((M2 @ H @ M1).T) ** 2
        cost += torch.norm(predict[i[1]] - y)
    return cost

In [8]:
seed = 42
torch.manual_seed(seed)
if device == "cuda":
    torch.cuda.manual_seed(seed)

x0 = nn.Parameter(torch.randn(36, dtype=torch.float32, device=device), requires_grad=True)
# optimizer = optim.Adam([x0], lr=0.3)
optimizer = torch_optimizer.Adahessian([x0], lr=0.3, weight_decay=1e-5, betas=(0.9, 0.999), eps=1e-8)
# U = create_un_mat(x0, device=device)
scheduler = CosineAnnealingLR(optimizer_x0, T_max=100)

print(x0)

Parameter containing:
tensor([ 0.1940,  2.1614, -0.1721,  0.8491, -1.9244,  0.6530, -0.6494, -0.8175,
         0.5280, -1.2753, -1.6621, -0.3033, -0.0926,  0.1992, -1.1204,  1.8577,
        -0.7145,  0.6881,  0.7968, -0.0334,  1.4917, -0.5165, -0.2541,  1.4746,
        -0.3260, -1.1600,  2.3551, -0.6924,  0.1837, -1.1835, -1.8029, -1.5808,
         0.8387,  1.4192,  0.6469,  0.4253], device='cuda:0',
       requires_grad=True)


In [7]:
x0_optimized = x0.clone()
print(x0_optimized)

tensor([ 0.1940,  2.1614, -0.1721,  0.8491, -1.9244,  0.6530, -0.6494, -0.8175,
         0.5280, -1.2753, -1.6621, -0.3033, -0.0926,  0.1992, -1.1204,  1.8577,
        -0.7145,  0.6881,  0.7968, -0.0334,  1.4917, -0.5165, -0.2541,  1.4746,
        -0.3260, -1.1600,  2.3551, -0.6924,  0.1837, -1.1835, -1.8029, -1.5808,
         0.8387,  1.4192,  0.6469,  0.4253], device='cuda:0',
       grad_fn=<CloneBackward0>)


In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/ex1 --port 6008

In [9]:
from torch.utils.tensorboard import SummaryWriter


min_loss = 1e10
steps = 100

seed = 42
torch.manual_seed(seed)
if device == "cuda":
    torch.cuda.manual_seed(seed)

lr_range = np.logspace(1, 0.1, 10)
T_max_range = np.linspace(10, 10000, 10)


def closure():
    optimizer.zero_grad()
    loss = f(x0, data)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(x0, max_norm=1.0)
    # print(f"loss: {loss.item()}")
    if loss.item() < 1e-5:
        print("Converged")
    return loss

# for lr in lr_range:
#     for T_max in T_max_range:

# run_id = datetime.now().strftime(f"lr={lr}_Tmax={T_max}")
run_id = datetime.now().strftime(f"lr=0.3_weight_decay=1e-5_betas=(0.9, 0.999)_eps=1e-8")
log_dir = os.path.join("logs", "ex1", run_id)
writer = SummaryWriter(log_dir=log_dir)

# optimizer = optim.Adam([x0], lr=lr)
# scheduler = CosineAnnealingLR(optimizer, T_max=T_max)

for n in range(steps):
    print(f"Step {n + 1}/{steps}")
    optimizer.step(closure)
    scheduler.step()
    #  print(x0.grad)

    with torch.no_grad():
        loss = f(x0, data)
        writer.add_scalar("Loss/train", loss.item(), n)
        writer.add_scalar("Learning rate", scheduler.get_last_lr()[0], n)
    
    #  if n % 100 == 0:  # Every 100 steps
    #     with torch.no_grad():
    #         perturbation = torch.randn_like(x0) * 0.01  # Small random noise
    #         x0.add_(perturbation)
    #         print(f"Applied random perturbation at step {n + 1}")

    #  print(f"Epoch {n+1}: Learning rate = {current_lr:.6f}")
    if n == steps - 1:
        print("Final loss:", closure().item())
        print("Final parameters:", x0.data.cpu().numpy())
        if loss.item() < min_loss:
            min_loss = loss.item()
            x0_optimized = x0.clone()
            print("Updated optimized parameters")
        x0 = nn.Parameter(torch.randn(36, dtype=torch.float32, device=device), requires_grad=True)

Step 1/100


RuntimeError: Gradient tensor 0 does not have grad_fn. When calling loss.backward(), make sure the option create_graph is set to True.

In [ ]:
writer.close()

import shutil
shutil.rmtree("logs/ex1", ignore_errors=True)

In [ ]:
print(x0)

In [ ]:
M1 = create_un_mat(x0[:12], device=device)
M2 = create_un_mat(x0[12:24], device=device)
alpha = torch.tensor(x0[24:33], dtype=torch.complex128, device=device).reshape(3, 3)
h_0 = torch.tensor(x0[33:36], dtype=torch.complex128, device=device)

fig, ax = plt.subplots(3, 4, figsize=(16, 10))

for j in range(12):

    for i in data[131*j:131*(j+1)]:
        x = torch.zeros(3, dtype=torch.complex128, device=device)
        x[i[0]] = 10**(-5) * i[2] ** 2
        y = torch.tensor(i[3:], dtype=torch.complex128, device=device)
        y /= torch.sum(y)

        h_list = h_0 + alpha @ x
        H_diag = torch.cat([torch.exp(1j * h_list), torch.tensor([1.0], dtype=torch.complex64, device=device)])
        H = torch.diag(H_diag)
        result = (torch.abs((M2 @ H @ M1).T) ** 2)[i[1]]
        ax[j//4][j%4].scatter(i[2], result[0].cpu().detach().numpy(), color = 'blue')
        ax[j//4][j%4].scatter(i[2], result[1].cpu().detach().numpy(), color = 'green')
        ax[j//4][j%4].scatter(i[2], result[2].cpu().detach().numpy(), color = 'yellow')
        ax[j//4][j%4].scatter(i[2], result[3].cpu().detach().numpy(), color = 'red')

    ax[j//4][j%4].plot([d[2] for d in data[131*j:131*(j+1)]], [d[3]/sum(d[3:]) for d in data[131*j:131*(j+1)]])#, color = 'blue')
    ax[j//4][j%4].plot([d[2] for d in data[131*j:131*(j+1)]], [d[4]/sum(d[3:]) for d in data[131*j:131*(j+1)]])#, color = 'green')
    ax[j//4][j%4].plot([d[2] for d in data[131*j:131*(j+1)]], [d[5]/sum(d[3:]) for d in data[131*j:131*(j+1)]])#, color = 'yellow')
    ax[j//4][j%4].plot([d[2] for d in data[131*j:131*(j+1)]], [d[6]/sum(d[3:]) for d in data[131*j:131*(j+1)]])#, color = 'red')

    ax[0][0].set_xlabel('Сила тока, единицы')
    ax[0][0].set_ylabel('Выходное излучение (норм)')

    fig.suptitle('Фитирование данных Ильи с помощью нашего кода')


plt.show()